In [1]:
!pip install yfinance -q
!pip install pandas -q
!pip install numpy -q
!pip install scipy -q

DEPRECATION: Loading egg at b:\my dear python projects\intepreter\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at b:\my dear python projects\intepreter\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at b:\my dear python projects\intepreter\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at b:\my dear python projects\intepreter\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [2]:
import time
import yfinance as yf
import pandas as pd
import numpy as np
import scipy.optimize as optimize
import scipy.interpolate as sci
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [3]:
def fetch_data(assets: list) -> pd.DataFrame:
    
    df = pd.DataFrame()
    
    for asset in assets:
        temp = yf.download(asset, start='2000-01-01', end='2023-08-27')
        
        temp.columns = [f"{asset}_{col}" for col in temp.columns]
        
        if df.empty:
            df = temp
        else:
            df = df.merge(temp, how='outer', left_index=True, right_index=True)
    
    df.dropna(inplace=True)
    
    return df

In [4]:
df = fetch_data (['TLT'])

[*********************100%%**********************]  1 of 1 completed


In [5]:
# Standardize data
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

# Convert to DataFrame
df_scaled = pd.DataFrame(df_scaled, columns=df.columns)

# Fill NaNs with last known values (ffill)
df_scaled.fillna(method='ffill', inplace=True)

In [6]:
df_scaled

,TLT_Open,TLT_High,TLT_Low,TLT_Close,TLT_Adj Close,TLT_Volume
0,-1.380586,-1.389783,-1.375842,-1.392767,-1.519215,-1.060270
1,-1.371095,-1.347311,-1.357705,-1.344813,-1.502892,-1.057019
2,-1.343098,-1.336930,-1.327159,-1.322499,-1.495296,-1.057633
3,-1.313677,-1.285964,-1.310931,-1.282142,-1.481558,-1.053754
4,-1.271919,-1.269919,-1.264634,-1.264575,-1.475578,-1.052596
...,...,...,...,...,...,...
5301,-0.862874,-0.877291,-0.864666,-0.870505,0.173555,3.020244
5302,-0.863824,-0.852752,-0.853688,-0.836795,0.196656,1.913756
5303,-0.785052,-0.745628,-0.769208,-0.727120,0.271815,4.281537
5304,-0.745666,-0.745628,-0.737707,-0.757031,0.251317,2.050159


In [8]:
df_scaled['Target'] = np.where(df_scaled['TLT_Adj Close'].shift(-1) < df_scaled['TLT_Adj Close'], 1, 0)

In [9]:
df_scaled

,TLT_Open,TLT_High,TLT_Low,TLT_Close,TLT_Adj Close,TLT_Volume,Target
0,-1.380586,-1.389783,-1.375842,-1.392767,-1.519215,-1.060270,0
1,-1.371095,-1.347311,-1.357705,-1.344813,-1.502892,-1.057019,0
2,-1.343098,-1.336930,-1.327159,-1.322499,-1.495296,-1.057633,0
3,-1.313677,-1.285964,-1.310931,-1.282142,-1.481558,-1.053754,0
4,-1.271919,-1.269919,-1.264634,-1.264575,-1.475578,-1.052596,1
...,...,...,...,...,...,...,...
5301,-0.862874,-0.877291,-0.864666,-0.870505,0.173555,3.020244,0
5302,-0.863824,-0.852752,-0.853688,-0.836795,0.196656,1.913756,0
5303,-0.785052,-0.745628,-0.769208,-0.727120,0.271815,4.281537,1
5304,-0.745666,-0.745628,-0.737707,-0.757031,0.251317,2.050159,0


In [10]:
X = df_scaled.drop('Target', axis=1)
y = df_scaled['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# LSTM

In [11]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    LSTM(64, return_sequences=True),
    LSTM(64, return_sequences=True),
    LSTM(32),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

X_train_lstm = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_lstm = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

model.fit(X_train_lstm, y_train, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10
120/120 [==============================] - 12s 32ms/step - loss: 0.6911 - accuracy: 0.5352 - val_loss: 0.6914 - val_accuracy: 0.5294
Epoch 2/10
120/120 [==============================] - 2s 19ms/step - loss: 0.6908 - accuracy: 0.5357 - val_loss: 0.6916 - val_accuracy: 0.5294
Epoch 3/10
120/120 [==============================] - 2s 20ms/step - loss: 0.6911 - accuracy: 0.5360 - val_loss: 0.6916 - val_accuracy: 0.5294
Epoch 4/10
120/120 [==============================] - 2s 19ms/step - loss: 0.6909 - accuracy: 0.5360 - val_loss: 0.6915 - val_accuracy: 0.5294
Epoch 5/10
120/120 [==============================] - 2s 18ms/step - loss: 0.6910 - accuracy: 0.5360 - val_loss: 0.6914 - val_accuracy: 0.5294
Epoch 6/10
120/120 [==============================] - 2s 18ms/step - loss: 0.6907 - accuracy: 0.5360 - val_loss: 0.6915 - val_accuracy: 0.5294
Epoch 7/10
120/120 [==============================] - 2s 20ms/step - loss: 0.6911 - accuracy: 0.5360 - val_loss: 0.6914 - val_accuracy: 0.529

In [12]:
X = df_scaled.drop('Target', axis=1)
y = df_scaled['Target']

In [13]:
y_pred = model.predict(X_test_lstm)
y_pred_binary = np.where(y_pred > 0.5, 1, 0)

accuracy = accuracy_score(y_test, y_pred_binary)
print("LSTM Accuracy:", accuracy)

34/34 [==============================] - 2s 6ms/step
LSTM Accuracy: 0.5018832391713748


# Naive

In [14]:
X = df_scaled.drop('Target', axis=1)
y = df_scaled['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
y_pred_naive = y_test.shift(1).fillna(0)

accuracy_naive = accuracy_score(y_test, y_pred_naive)
print("Naive Approach Accuracy:", accuracy_naive)

Naive Approach Accuracy: 0.4839924670433145
